In [1]:
import torch
import clip
from PIL import Image
import logging
import pandas as pd
import glob 
import os
from io import BytesIO
from PIL import UnidentifiedImageError
import subprocess
from weat.test import Test
import torch
import clip
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device("cuda:2")
print(device)
model, preprocess = clip.load("ViT-L/14", device=device)

class dwebpException(Exception):
    pass

def dwebp(file: str):
    webp = subprocess.run(
        f"dwebp  {file} -quiet -o -", shell=True, capture_output=True
    )
    if webp.returncode != 0:
        raise dwebpException(webp.stderr.decode())
    else:
        return Image.open(BytesIO(webp.stdout))

def load_dir(path):
    tmp = []
    
    for file in glob.glob(path):
        # 파일 확장자가 .json이면 무시
        if os.path.splitext(file)[1].lower() == ".json":
            continue

        with torch.no_grad():
            try:
                img = Image.open(file)
            except UnidentifiedImageError:
                if os.path.splitext(file)[1].lower() == ".webp":
                    img = dwebp(file)
                  
                else:
                    raise
            prep = preprocess(img).unsqueeze(0).to(device)
            emb = model.encode_image(prep)
            tmp.append(emb.cpu())
    return tmp

def ieat_calc(X_Image_Paths, Y_Image_Paths, A_texts, B_texts, Names):
    df = pd.DataFrame(columns=['Name', 'X', 'Y', 'A', 'B', 'n_t', 'n_a', 'p_i', 'd_i'])
    for i in range(len(Names[0])):
        X_image = torch.cat(load_dir(f'{X_Image_Paths[i]}*'))
        Y_image = torch.cat(load_dir(f'{Y_Image_Paths[i]}*'))

        with torch.no_grad():
            A_text = model.encode_text(A_texts[i]).to("cpu")
            B_text = model.encode_text(B_texts[i]).to("cpu")
                
        test = Test(X_image, Y_image, A_text, B_text)
        out = test.run()
        result = {
            'Name': Names[0][i],
            'X': Names[1][i],
            'Y': Names[2][i],
            'A': Names[3][i],
            'B': Names[4][i],
            'n_t': X_image.shape[0],
            'n_a': A_text.shape[0], 
            'p_i': out[1],
            'd_i': out[0]
        }
        df = df.append(result, ignore_index=True)
    return df

import json
import torch
import clip

def load_caption(load_dir, model, device):
    # 1. JSON 파일 읽기
    with open(load_dir+'captions.json', 'r') as f:
        data = json.load(f)

    # 2. "caption" 키의 값만 추출
    captions = [item['caption'] for item in data]

    return model.encode_text(clip.tokenize(captions).to(device))


/home/bubble3jh/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda:2


In [3]:
class dwebpException(Exception):
    pass

def dwebp(file: str):
    webp = subprocess.run(
        f"dwebp  {file} -quiet -o -", shell=True, capture_output=True
    )
    if webp.returncode != 0:
        raise dwebpException(webp.stderr.decode())
    else:
        return Image.open(BytesIO(webp.stdout))

def load_dir(path):
    tmp = []
    
    for file in glob.glob(path):
        # 파일 확장자가 .json이면 무시
        if os.path.splitext(file)[1].lower() == ".json":
            continue

        with torch.no_grad():
            try:
                img = Image.open(file)
            except UnidentifiedImageError:
                if os.path.splitext(file)[1].lower() == ".webp":
                    img = dwebp(file)
                  
                else:
                    raise
            prep = preprocess(img).unsqueeze(0).to(device)
            emb = model.encode_image(prep)
            tmp.append(emb.cpu())
    return tmp

In [4]:
def ieat_calc(X_Image_Paths, Y_Image_Paths, A_texts, B_texts, Names):
    df = pd.DataFrame(columns=['Name', 'X', 'Y', 'A', 'B', 'n_t', 'n_a', 'p_i', 'd_i'])
    for i in range(len(Names[0])):
        X_image = torch.cat(load_dir(f'{X_Image_Paths[i]}*'))
        Y_image = torch.cat(load_dir(f'{Y_Image_Paths[i]}*'))

        with torch.no_grad():
            A_text = model.encode_text(A_texts[i]).to("cpu")
            B_text = model.encode_text(B_texts[i]).to("cpu")
                
        test = Test(X_image, Y_image, A_text, B_text)
        out = test.run()
        result = {
            'Name': Names[0][i],
            'X': Names[1][i],
            'Y': Names[2][i],
            'A': Names[3][i],
            'B': Names[4][i],
            'n_t': X_image.shape[0],
            'n_a': A_text.shape[0], 
            'p_i': out[1],
            'd_i': out[0]
        }
        df = df.append(result, ignore_index=True)
    return df

In [5]:
import json
import torch
import clip

def load_caption(load_dir, model, device):
    # 1. JSON 파일 읽기
    with open(load_dir+'captions.json', 'r') as f:
        data = json.load(f)

    # 2. "caption" 키의 값만 추출
    captions = [item['caption'] for item in data]

    return model.encode_text(clip.tokenize(captions).to(device))


### Naive image-caption bias

In [2]:
X_image = torch.cat(load_dir('./gender/male/*'))
Y_image = torch.cat(load_dir('./gender/female/*'))

A_text = load_caption('./gender/science/', model, device).detach().to("cpu")
B_text = load_caption('./gender/liberal-arts/', model, device).detach().to("cpu")

test = Test(X_image, Y_image, A_text, B_text)
out = test.run()

df = pd.DataFrame([{'Name': 'Gender', 'X': 'Male', 'Y': 'Female', 'A': 'Science', 'B': 'Arts', 'n_t': X_image.shape[0], 'n_a': A_text.shape[0], 'p_i': out[1], 'd_i': out[0]}])
df

11/02 11:26:30 PM: Computing cosine similarities...
11/02 11:26:30 PM: Null hypothesis: no difference between X and Y in association to attributes A and B
11/02 11:26:30 PM: Computing pval...
11/02 11:26:30 PM: Using non-parametric test
11/02 11:26:30 PM: Drawing 9999 samples (and biasing by 1)
11/02 11:26:30 PM: pval: 0.0096
11/02 11:26:30 PM: computing effect size...
11/02 11:26:30 PM: esize: 0.445199


,Name,X,Y,A,B,n_t,n_a,p_i,d_i
0,Gender,Male,Female,Science,Arts,55,55,0.0096,0.445199


### Swapped image-caption bias

In [2]:
# X_image = torch.cat(load_dir('./gender/male/*'))
# Y_image = torch.cat(load_dir('./gender/female/*'))

# A_text = load_caption('./gender/science/', model, device).detach().to("cpu")
# B_text = load_caption('./gender/liberal-arts/', model, device).detach().to("cpu")

X_image = torch.cat(load_dir('./gender/science/*'))
Y_image = torch.cat(load_dir('./gender/liberal-arts/*'))

A_text = load_caption('./gender/male/', model, device).detach().to("cpu")
B_text = load_caption('./gender/female/', model, device).detach().to("cpu")

test = Test(X_image, Y_image, A_text, B_text)
out = test.run()

df = pd.DataFrame([{'Name': 'Gender', 'X': 'Male', 'Y': 'Female', 'A': 'Science', 'B': 'Arts', 'n_t': X_image.shape[0], 'n_a': A_text.shape[0], 'p_i': out[1], 'd_i': out[0]}])
df

11/02 11:16:21 PM: Computing cosine similarities...
11/02 11:16:21 PM: Null hypothesis: no difference between X and Y in association to attributes A and B
11/02 11:16:21 PM: Computing pval...
11/02 11:16:21 PM: Using non-parametric test
11/02 11:16:21 PM: Drawing 9999 samples (and biasing by 1)
11/02 11:16:21 PM: pval: 0.0034
11/02 11:16:21 PM: computing effect size...
11/02 11:16:21 PM: esize: 0.514824


,Name,X,Y,A,B,n_t,n_a,p_i,d_i
0,Gender,Male,Female,Science,Arts,55,55,0.0034,0.514824


In [6]:
import torch
import pandas as pd

# Presuming load_dir and load_caption are defined functions that load the images and captions
# and Test is a class that has been defined to take images and captions and run some kind of test

# The pairs as provided
pairs = [
  "gender/male:gender/female:gender/science:gender/liberal-arts",
  "intersectional/white-male:intersectional/black-female:gender/science:gender/liberal-arts",
  "arab-muslim/other-people:arab-muslim/arab-muslim:valence/pleasant:valence/unpleasant",
  "american_race/european-american-google:american_race/african-american-google:valence/pleasant:valence/unpleasant",
  "american_race/european-american-google:american_race/asian-american-google:valence/pleasant:valence/unpleasant",
  "sexuality/straight:sexuality/gay:valence/pleasant:valence/unpleasant",
]

# Prepare a list to hold DataFrame rows
df_rows = []

# Iterate over the pairs to perform tests and collect results
for pair_string in pairs:
    # Split each pair string into its respective parts
    A_attribute, B_attribute, cap1, cap2 = pair_string.split(':')

    # Load images
    X_images = torch.cat(load_dir(f'./{cap1}/*'))
    Y_images = torch.cat(load_dir(f'./{cap2}/*'))

    # Load captions
    A_text = load_caption(f'./{A_attribute}/', model, device).detach().to("cpu")
    B_text = load_caption(f'./{B_attribute}/', model, device).detach().to("cpu")

    # Perform the test
    test = Test(X_images, Y_images, A_text, B_text)
    out = test.run()

    # Create a dictionary for the current row and append to the list
    df_row = {
        'A_attribute': A_attribute,
        'B_attribute': B_attribute,
        'cap1': cap1,
        'cap2': cap2,
        'n_X_images': X_images.shape[0],
        'n_Y_images': Y_images.shape[0],
        'n_A_text': A_text.shape[0],
        'n_B_text': B_text.shape[0],
        'test_result': out  # Assuming 'out' is a result that can be stored directly; adjust as needed
    }
    df_rows.append(df_row)

# Create the DataFrame
df = pd.DataFrame(df_rows)

# Display or save the DataFrame as needed
print(df)
df.to_csv('swapped_biased_result.csv', index=False)


11/02 11:47:44 PM: Computing cosine similarities...
11/02 11:47:44 PM: Null hypothesis: no difference between X and Y in association to attributes A and B
11/02 11:47:44 PM: Computing pval...
11/02 11:47:44 PM: Using non-parametric test
11/02 11:47:44 PM: Drawing 9999 samples (and biasing by 1)
11/02 11:47:44 PM: pval: 0.0031
11/02 11:47:44 PM: computing effect size...
11/02 11:47:44 PM: esize: 0.514824
11/02 11:47:53 PM: Computing cosine similarities...
11/02 11:47:53 PM: Null hypothesis: no difference between X and Y in association to attributes A and B
11/02 11:47:53 PM: Computing pval...
11/02 11:47:53 PM: Using non-parametric test
11/02 11:47:53 PM: Drawing 9999 samples (and biasing by 1)
11/02 11:47:53 PM: pval: 0.0029
11/02 11:47:53 PM: computing effect size...
11/02 11:47:53 PM: esize: 0.502419
11/02 11:48:00 PM: Computing cosine similarities...
11/02 11:48:00 PM: Null hypothesis: no difference between X and Y in association to attributes A and B
11/02 11:48:00 PM: Computing pv

                              A_attribute  \
0                             gender/male   
1               intersectional/white-male   
2                arab-muslim/other-people   
3  american_race/european-american-google   
4  american_race/european-american-google   
5                      sexuality/straight   

                             B_attribute              cap1  \
0                          gender/female    gender/science   
1            intersectional/black-female    gender/science   
2                arab-muslim/arab-muslim  valence/pleasant   
3  american_race/african-american-google  valence/pleasant   
4    american_race/asian-american-google  valence/pleasant   
5                          sexuality/gay  valence/pleasant   

                  cap2  n_X_images  n_Y_images  n_A_text  n_B_text  \
0  gender/liberal-arts          55          55        55        55   
1  gender/liberal-arts          55          55        55        55   
2   valence/unpleasant          55     

### FarconVAE Neutralized image-caption bias

In [9]:
import torch

X_image = torch.cat(load_dir('./gender/science/*'))
Y_image = torch.cat(load_dir('./gender/liberal-arts/*'))

A_text = torch.load('/data1/bubble3jh/farcon/git_FarconVAE/neut_embeddings/imgs/gender/male_ViT-L14.pt').detach().to("cpu")
B_text = torch.load('/data1/bubble3jh/farcon/git_FarconVAE/neut_embeddings/imgs/gender/female_ViT-L14.pt').detach().to("cpu")

test = Test(X_image, Y_image, A_text, B_text)
out = test.run()

df = pd.DataFrame([{'Name': 'Gender', 'X': 'Male', 'Y': 'Female', 'A': 'Science', 'B': 'Arts', 'n_t': X_image.shape[0], 'n_a': A_text.shape[0], 'p_i': out[1], 'd_i': out[0]}])
df

11/02 11:50:29 PM: Computing cosine similarities...
11/02 11:50:29 PM: Null hypothesis: no difference between X and Y in association to attributes A and B
11/02 11:50:29 PM: Computing pval...
11/02 11:50:29 PM: Using non-parametric test
11/02 11:50:29 PM: Drawing 9999 samples (and biasing by 1)
11/02 11:50:29 PM: pval: 0.1421
11/02 11:50:29 PM: computing effect size...
11/02 11:50:29 PM: esize: 0.206255


,Name,X,Y,A,B,n_t,n_a,p_i,d_i
0,Gender,Male,Female,Science,Arts,55,15,0.1421,0.206255


In [13]:
import torch
import pandas as pd

# Assuming load_dir and Test are defined functions/classes
# and the embeddings for A_text and B_text are stored at specified paths

# The pairs as provided
pairs = [
  "gender/male:gender/female:gender/science:gender/liberal-arts",
  "intersectional/white-male:intersectional/black-female:gender/science:gender/liberal-arts",
  "arab-muslim/other-people:arab-muslim/arab-muslim:valence/pleasant:valence/unpleasant",
  "american_race/european-american-google:american_race/african-american-google:valence/pleasant:valence/unpleasant",
  "american_race/european-american-google:american_race/asian-american-google:valence/pleasant:valence/unpleasant",
  "sexuality/straight:sexuality/gay:valence/pleasant:valence/unpleasant",
]

# Prepare a list to hold DataFrame rows
df_rows = []

# Path to the neutral embeddings
neut_embedding_base_path = '/data1/bubble3jh/farcon/git_FarconVAE/neut_embeddings/imgs/'

# Iterate over the pairs to perform tests and collect results
for pair_string in pairs:
    # Split each pair string into its respective parts
    A_attribute, B_attribute, cap1, cap2 = pair_string.split(':')

    # Load images
    X_images = torch.cat(load_dir(f'./{cap1}/*'))
    Y_images = torch.cat(load_dir(f'./{cap2}/*'))

    # Load precomputed embeddings for A_text and B_text
    A_embedding_path = f"{neut_embedding_base_path}{A_attribute}_ViT-L14.pt"
    B_embedding_path = f"{neut_embedding_base_path}{B_attribute}_ViT-L14.pt"
    A_text = torch.load(A_embedding_path).detach().to("cpu")
    B_text = torch.load(B_embedding_path).detach().to("cpu")

    # Perform the test
    test = Test(X_images, Y_images, A_text, B_text)
    out = test.run()

    # Create a dictionary for the current row and append to the list
    df_row = {
        'Name': A_attribute + ' vs. ' + B_attribute,
        'X': cap1.split('/')[1],
        'Y': cap2.split('/')[1],
        'A': A_attribute.split('/')[1],
        'B': B_attribute.split('/')[1],
        'n_t': X_images.shape[0],
        'n_a': A_text.shape[0],
        'p_i': out[1],  # Assuming this is a scalar
        'd_i': out[0]   # Assuming this is a scalar
    }
    df_rows.append(df_row)

# Create the DataFrame
df = pd.DataFrame(df_rows)

# Display or save the DataFrame as needed
print(df)
df.to_csv('swapped_neuted_result.csv', index=False)


11/03 12:13:24 AM: Computing cosine similarities...
11/03 12:13:24 AM: Null hypothesis: no difference between X and Y in association to attributes A and B
11/03 12:13:24 AM: Computing pval...
11/03 12:13:24 AM: Using non-parametric test
11/03 12:13:24 AM: Drawing 9999 samples (and biasing by 1)
11/03 12:13:24 AM: pval: 0.1406
11/03 12:13:24 AM: computing effect size...
11/03 12:13:24 AM: esize: 0.206255
11/03 12:13:30 AM: Computing cosine similarities...
11/03 12:13:30 AM: Null hypothesis: no difference between X and Y in association to attributes A and B
11/03 12:13:30 AM: Computing pval...
11/03 12:13:30 AM: Using non-parametric test
11/03 12:13:30 AM: Drawing 9999 samples (and biasing by 1)
11/03 12:13:30 AM: pval: 0.1472
11/03 12:13:30 AM: computing effect size...
11/03 12:13:30 AM: esize: 0.206256
11/03 12:13:35 AM: Computing cosine similarities...
11/03 12:13:35 AM: Null hypothesis: no difference between X and Y in association to attributes A and B
11/03 12:13:35 AM: Computing pv

                                                Name         X             Y  \
0                      gender/male vs. gender/female   science  liberal-arts   
1  intersectional/white-male vs. intersectional/b...   science  liberal-arts   
2  arab-muslim/other-people vs. arab-muslim/arab-...  pleasant    unpleasant   
3  american_race/european-american-google vs. ame...  pleasant    unpleasant   
4  american_race/european-american-google vs. ame...  pleasant    unpleasant   
5               sexuality/straight vs. sexuality/gay  pleasant    unpleasant   

                          A                        B  n_t  n_a     p_i  \
0                      male                   female   55   15  0.1406   
1                white-male             black-female   55   15  0.1472   
2              other-people              arab-muslim   55   10  0.0001   
3  european-american-google  african-american-google   55   39  0.4090   
4  european-american-google    asian-american-google   55   39  0.000

In [15]:
X_image = torch.cat(load_dir('./ieat/data/experiments/gender/male/*'))
Y_image = torch.cat(load_dir('./ieat/data/experiments/gender/female/*'))
A_image = torch.cat(load_dir('./ieat/data/experiments/gender/engineering/*'))
B_image = torch.cat(load_dir('./ieat/data/experiments/gender/care/*'))

test = Test(X_image, Y_image, A_image, B_image)
out = test.run()
        
df = pd.DataFrame([{'Name': 'Gender', 'X': 'Male', 'Y': 'Female', 'A': 'Engineering', 'B': 'Caregiving', 'n_t': X_image.shape[0], 'n_a': A_image.shape[0], 'p_i': out[1], 'd_i': out[0]}])
df

RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [8]:
X_image = torch.cat(load_dir('./gender/male/*'))
Y_image = torch.cat(load_dir('./gender/female/*'))
A_image = torch.cat(load_dir('./gender/career/*'))
B_image = torch.cat(load_dir('./gender/family/*'))

test = Test(X_image, Y_image, A_image, B_image)
out = test.run()
        
df = pd.DataFrame([{'Name': 'Gender', 'X': 'Male', 'Y': 'Female', 'A': 'Career', 'B': 'Family', 'n_t': X_image.shape[0], 'n_a': A_image.shape[0], 'p_i': out[1], 'd_i': out[0]}])
df

NameError: name 'preprocess' is not defined

In [ ]:
X_image = torch.cat(load_dir('./ieat/data/experiments/race/european-american-male/*'))
Y_image = torch.cat(load_dir('./ieat/data/experiments/race/african-american-female/*'))
A_image = torch.cat(load_dir('./ieat/data/experiments/gender/science/*'))
B_image = torch.cat(load_dir('./ieat/data/experiments/gender/liberal-arts/*'))

test = Test(X_image, Y_image, A_image, B_image)
out = test.run()
        
df = pd.DataFrame([{'Name': 'Ethnicity', 'X': 'African-American', 'Y': 'European-American', 'A': 'Science', 'B': 'Arts', 'n_t': X_image.shape[0], 'n_a': A_image.shape[0], 'p_i': out[1], 'd_i': out[0]}])
df

In [ ]:
X_image = torch.cat(load_dir('./ieat/data/experiments/race/european-american-male/*'))
Y_image = torch.cat(load_dir('./ieat/data/experiments/race/african-american-female/*'))
A_image = torch.cat(load_dir('./ieat/data/experiments/gender/engineering/*'))
B_image = torch.cat(load_dir('./ieat/data/experiments/gender/care/*'))

test = Test(X_image, Y_image, A_image, B_image)
out = test.run()
        
df = pd.DataFrame([{'Name': 'Ethnicity', 'X': 'African-American', 'Y': 'European-American', 'A': 'Engineering', 'B': 'Caregiving', 'n_t': X_image.shape[0], 'n_a': A_image.shape[0], 'p_i': out[1], 'd_i': out[0]}])
df

In [ ]:
X_image = torch.cat(load_dir('./ieat/data/experiments/race/european-american-male/*'))
Y_image = torch.cat(load_dir('./ieat/data/experiments/race/african-american-female/*'))
A_image = torch.cat(load_dir('./ieat/data/experiments/gender/career/*'))
B_image = torch.cat(load_dir('./ieat/data/experiments/gender/family/*'))

test = Test(X_image, Y_image, A_image, B_image)
out = test.run()
        
df = pd.DataFrame([{'Name': 'Ethnicity', 'X': 'African-American', 'Y': 'European-American', 'A': 'Career', 'B': 'Family', 'n_t': X_image.shape[0], 'n_a': A_image.shape[0], 'p_i': out[1], 'd_i': out[0]}])
df

In [ ]:
X_image = torch.cat(load_dir('./ieat/data/experiments/arab-muslim/other-people/*'))
Y_image = torch.cat(load_dir('./ieat/data/experiments/arab-muslim/arab-muslim/*'))
A_image = torch.cat(load_dir('./ieat/data/experiments/valence/pleasant/*'))
B_image = torch.cat(load_dir('./ieat/data/experiments/valence/unpleasant/*'))

test = Test(X_image, Y_image, A_image, B_image)
out = test.run()
        
df = pd.DataFrame([{'Name': 'Ethnicity', 'X': 'other-people', 'Y': 'arab-muslim', 'A': 'pleasant', 'B': 'unpleasant', 'n_t': X_image.shape[0], 'n_a': A_image.shape[0], 'p_i': out[1], 'd_i': out[0]}])
df

In [ ]:
X_image = torch.cat(load_dir('./ieat/data/experiments/race/european-american/*'))
Y_image = torch.cat(load_dir('./ieat/data/experiments/race/african-american/*'))
A_image = torch.cat(load_dir('./ieat/data/experiments/valence/pleasant/*'))
B_image = torch.cat(load_dir('./ieat/data/experiments/valence/unpleasant/*'))

test = Test(X_image, Y_image, A_image, B_image)
out = test.run()
        
df = pd.DataFrame([{'Name': 'Ethnicity', 'X': 'european-american', 'Y': 'african-american', 'A': 'pleasant', 'B': 'unpleasant', 'n_t': X_image.shape[0], 'n_a': A_image.shape[0], 'p_i': out[1], 'd_i': out[0]}])
df

In [ ]:
X_image = torch.cat(load_dir('./ieat/data/experiments/asian/european-american/*'))
Y_image = torch.cat(load_dir('./ieat/data/experiments/asian/asian-american/*'))
A_image = torch.cat(load_dir('./ieat/data/experiments/valence/pleasant/*'))
B_image = torch.cat(load_dir('./ieat/data/experiments/valence/unpleasant/*'))

test = Test(X_image, Y_image, A_image, B_image)
out = test.run()
        
df = pd.DataFrame([{'Name': 'Ethnicity', 'X': 'european-american', 'Y': 'asian-american', 'A': 'pleasant', 'B': 'unpleasant', 'n_t': X_image.shape[0], 'n_a': A_image.shape[0], 'p_i': out[1], 'd_i': out[0]}])
df

In [ ]:
X_image = torch.cat(load_dir('./ieat/data/experiments/weapon/white/*'))
Y_image = torch.cat(load_dir('./ieat/data/experiments/weapon/black/*'))
A_image = torch.cat(load_dir('./ieat/data/experiments/valence/pleasant/*'))
B_image = torch.cat(load_dir('./ieat/data/experiments/valence/unpleasant/*'))

test = Test(X_image, Y_image, A_image, B_image)
out = test.run()
        
df = pd.DataFrame([{'Name': 'Ethnicity', 'X': 'white', 'Y': 'black', 'A': 'pleasant', 'B': 'unpleasant', 'n_t': X_image.shape[0], 'n_a': A_image.shape[0], 'p_i': out[1], 'd_i': out[0]}])
df

In [ ]:
X_image = torch.cat(load_dir('./ieat/data/experiments/weapon/white/*'))
Y_image = torch.cat(load_dir('./ieat/data/experiments/weapon/black/*'))
A_image = torch.cat(load_dir('./ieat/data/experiments/weapon/tool-modern/*'))
B_image = torch.cat(load_dir('./ieat/data/experiments/weapon/weapon-modern/*'))

test = Test(X_image, Y_image, A_image, B_image)
out = test.run()
        
df = pd.DataFrame([{'Name': 'Ethnicity', 'X': 'white', 'Y': 'black', 'A': 'tool', 'B': 'weapon', 'n_t': X_image.shape[0], 'n_a': A_image.shape[0], 'p_i': out[1], 'd_i': out[0]}])
df

In [ ]:
X_image = torch.cat(load_dir('./ieat/data/experiments/weapon/white/*'))
Y_image = torch.cat(load_dir('./ieat/data/experiments/weapon/black/*'))
A_image = torch.cat(load_dir('./ieat/data/experiments/weapon/tool/*'))
B_image = torch.cat(load_dir('./ieat/data/experiments/weapon/weapon/*'))

test = Test(X_image, Y_image, A_image, B_image)
out = test.run()
        
df = pd.DataFrame([{'Name': 'Ethnicity', 'X': 'white', 'Y': 'black', 'A': 'tool', 'B': 'weapon', 'n_t': X_image.shape[0], 'n_a': A_image.shape[0], 'p_i': out[1], 'd_i': out[0]}])
df

In [ ]:
X_image = torch.cat(load_dir('./ieat/data/experiments/skin-tone/light/*'))
Y_image = torch.cat(load_dir('./ieat/data/experiments/skin-tone/dark/*'))
A_image = torch.cat(load_dir('./ieat/data/experiments/weapon/tool/*'))
B_image = torch.cat(load_dir('./ieat/data/experiments/weapon/weapon/*'))

test = Test(X_image, Y_image, A_image, B_image)
out = test.run()
        
df = pd.DataFrame([{'Name': 'Ethnicity', 'X': 'white', 'Y': 'black', 'A': 'tool', 'B': 'weapon', 'n_t': X_image.shape[0], 'n_a': A_image.shape[0], 'p_i': out[1], 'd_i': out[0]}])
df

In [ ]:
X_image = torch.cat(load_dir('./ieat/data/experiments/sexuality/straight/*'))
Y_image = torch.cat(load_dir('./ieat/data/experiments/sexuality/gay/*'))
A_image = torch.cat(load_dir('./ieat/data/experiments/valence/pleasant/*'))
B_image = torch.cat(load_dir('./ieat/data/experiments/valence/unpleasant/*'))

test = Test(X_image, Y_image, A_image, B_image)
out = test.run()
        
df = pd.DataFrame([{'Name': 'Ethnicity', 'X': 'straight', 'Y': 'gay', 'A': 'pleasant', 'B': 'unpleasant', 'n_t': X_image.shape[0], 'n_a': A_image.shape[0], 'p_i': out[1], 'd_i': out[0]}])
df